In [0]:
import numpy as np
import pandas as pd
import os
import pickle
from sklearn.externals import joblib
import sklearn
import sklearn.model_selection
from sklearn.naive_bayes import GaussianNB
from google.colab import drive
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier 
from sklearn import preprocessing
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functio

In [0]:
drive.mount('/content/drive')
path = os.getcwd()
df = pd.DataFrame()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df = pd.DataFrame()
#print(path)
weather_df = pd.read_csv(path + '/drive/My Drive/ML Project/Weather.csv')
flight_df = pd.read_csv(path + '/drive/My Drive/ML Project/Flight.csv')


In [0]:
weather_df.columns

Index(['windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM',
       'visibility', 'pressure', 'cloudcover', 'DewPointF', 'WindGustKmph',
       'tempF', 'WindChillF', 'humidity', 'date', 'time', 'airport'],
      dtype='object')

In [0]:
# Rounding off military time to the nearest hour
rem = 0
def round_off(data):
  if(data>=2300):
    return(2300)
  else:
    rem = data%100
    if (rem>=30):
      data = data - rem
      data = data +100
    else:
      return(data - rem)
  return(data)
flight_df["CRSDepTimeRound"] = pd.Series(map(lambda x: round_off(x), flight_df["CRSDepTime"]))


In [0]:
# Merging the Fight and Weather Datasets
df = pd.merge(flight_df, weather_df, how = 'left', left_on = ['Origin','FlightDate', 'CRSDepTimeRound'],right_on = ['airport', 'date', 'time'])
df.dropna(inplace = True)

KeyError: ignored

In [0]:
#df.to_csv(path + '/drive/My Drive/ML Project/Merge.csv', index = False)

NameError: ignored

In [0]:
df = pd.read_csv(path + '/drive/My Drive/ML Project/Merge.csv')

train, test, train_labels, test_labels = sklearn.model_selection.train_test_split(df.drop(['airport'],axis = 1),df['ArrDel15'], test_size = 0.25, random_state = 3)

In [0]:
# One-hot Encoding
OriginOHE = pd.get_dummies(train['Origin'])
DestOHE = pd.get_dummies(train['Dest'])
OriginOHE_test = pd.get_dummies(test['Origin'])
DestOHE_test = pd.get_dummies(test['Dest'])

train.join(OriginOHE)
train.join(DestOHE)
test.join(OriginOHE_test)
test.join(DestOHE_test)

,FlightDate,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,Origin,Dest,ArrTime,CRSArrTime,ArrDel15,ArrDelayMinutes,CRSDepTimeRound,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,WindChillF,humidity,date,time,ATL,CLT,DEN,DFW,EWR,IAH,JFK,LAS,LAX,MCO,MIA,ORD,PHX,SEA,SFO
1843590,2017-12-08,4,2017,12,8,734.0,0.0,735,0.0,DFW,IAH,846.0,850,0.0,0.0,800,9,319,113,0.0,10,1026,0,19,10,33,30,55,2017-12-08,800,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
867469,2016-12-07,4,2016,12,7,1151.0,0.0,1200,0.0,SFO,PHX,1436.0,1457,0.0,0.0,1200,12,104,122,0.0,10,1022,100,40,14,50,47,68,2016-12-07,1200,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
977447,2017-01-20,1,2017,1,20,718.0,0.0,720,0.0,DFW,ATL,1018.0,1025,0.0,0.0,700,14,213,113,0.0,10,1003,6,46,29,49,45,92,2017-01-20,700,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1271977,2017-05-17,2,2017,5,17,558.0,0.0,605,0.0,SEA,PHX,839.0,845,0.0,0.0,600,10,177,116,0.0,5,1017,61,45,12,48,49,86,2017-05-17,600,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1592990,2017-09-07,3,2017,9,7,1820.0,1.0,1759,21.0,CLT,MCO,2020.0,1943,1.0,37.0,1800,6,260,113,0.0,10,1017,0,53,7,75,75,47,2017-09-07,1800,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533665,2016-07-22,3,2016,7,22,1323.0,0.0,1333,0.0,LAS,DEN,1611.0,1628,0.0,0.0,1400,8,169,113,0.0,10,1010,0,38,9,109,108,9,2016-07-22,1400,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1281425,2017-05-09,2,2017,5,9,1354.0,0.0,1352,2.0,SFO,DFW,1940.0,1930,0.0,10.0,1400,22,281,113,0.0,10,1009,0,47,24,67,68,48,2017-05-09,1400,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
882762,2016-12-09,4,2016,12,9,1706.0,0.0,1715,0.0,MCO,DFW,1859.0,1916,0.0,0.0,1700,20,8,122,0.0,10,1026,52,45,25,60,58,59,2016-12-09,1700,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1177178,2017-04-20,2,2017,4,20,837.0,0.0,845,0.0,LAS,MIA,1619.0,1629,0.0,0.0,900,10,269,113,0.0,10,1013,67,19,12,65,67,16,2017-04-20,900,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [0]:
train.to_csv(path + "/drive/My Drive/ML Project/Train.csv", index = False)
test.to_csv(path + "/drive/My Drive/ML Project/Test.csv", index = False)
print(train.shape,test.shape)

(1388577, 30) (462859, 30)


In [0]:
undersample = RandomUnderSampler(sampling_strategy='majority', random_state=1)
train_under, train_labels_under = undersample.fit_resample(train.drop(['Origin','Dest','FlightDate','date', 'ArrDelayMinutes','ArrDel15','ArrTime'], axis = 1), train_labels)
#oversample = SMOTE()
#train_under, train_labels_under = oversample.fit_resample(train.drop(['Origin','Dest','FlightDate','date', 'ArrDelayMinutes','ArrDel15','ArrTime'], axis = 1), train['ArrDel15'])
#print(type(train_under))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [0]:
#Using GNB 

gnb = GaussianNB()

#gnb.fit(train_u.drop(['ArrDel15', 'ArrTime'], axis = 1), train_labels_u)
gnb.fit(train_under, train_labels_under)
prediction_gnb = gnb.predict(test.drop(['Origin','Dest','FlightDate','date', 'ArrDelayMinutes', 'ArrDel15','ArrTime'], axis = 1))
print(classification_report(test_labels, prediction_gnb))
filename = path + '/drive/My Drive/ML Project/classifier_model_gnb_ru.sav'
pickle.dump(gnb, open(filename, 'wb'))

              precision    recall  f1-score   support

         0.0       0.94      0.93      0.93    365677
         1.0       0.73      0.77      0.75     97182

    accuracy                           0.89    462859
   macro avg       0.83      0.85      0.84    462859
weighted avg       0.89      0.89      0.89    462859



In [0]:
#Using RandomForest

clf = RandomForestClassifier(n_estimators = 100)
#clf.fit(train.drop(['Origin','Dest','FlightDate','date', 'ArrDelayMinutes', 'ArrDel15','ArrTime'], axis = 1), train_labels)
clf.fit(train_under, train_labels_under)
prediction_rf = clf.predict(test.drop(['Origin','Dest','FlightDate','date', 'ArrDelayMinutes', 'ArrDel15','ArrTime'], axis = 1))


In [0]:
print(classification_report(test_labels, prediction_rf))
filename = path + '/drive/My Drive/ML Project/classifier_model_rf_ru.sav'
pickle.dump(clf, open(filename, 'wb'))

              precision    recall  f1-score   support

         0.0       0.93      0.96      0.95    365677
         1.0       0.83      0.72      0.78     97182

    accuracy                           0.91    462859
   macro avg       0.88      0.84      0.86    462859
weighted avg       0.91      0.91      0.91    462859



In [0]:
#Using ExtraTrees Classifier

extra_tree_forest = ExtraTreesClassifier(n_estimators = 5)
#extra_tree_forest.fit(train.drop(['Origin','Dest','FlightDate','date', 'ArrDelayMinutes', 'ArrDel15','ArrTime'], axis = 1), train_labels)
extra_tree_forest.fit(train_under, train_labels_under)
prediction_et = extra_tree_forest.predict((test.drop(['Origin','Dest','FlightDate','date', 'ArrDelayMinutes', 'ArrDel15','ArrTime'], axis = 1)))

In [0]:
print(classification_report(test_labels, prediction_et))
filename = path + '/drive/My Drive/ML Project/classifier_model_et_ru.sav'
pickle.dump(extra_tree_forest, open(filename, 'wb'))

              precision    recall  f1-score   support

         0.0       0.93      0.93      0.93    365677
         1.0       0.74      0.73      0.73     97182

    accuracy                           0.89    462859
   macro avg       0.83      0.83      0.83    462859
weighted avg       0.89      0.89      0.89    462859



In [0]:
#Using LogisticRegression Classifier
logreg = LogisticRegression()
#logreg.fit(train.drop(['Origin','Dest','FlightDate','date', 'ArrDelayMinutes', 'ArrDel15', 'ArrTime'], axis = 1), train_labels)
logreg.fit(train_under, train_labels_under)
prediction_lr = logreg.predict((test.drop(['Origin','Dest','FlightDate','date', 'ArrDelayMinutes','ArrDel15', 'ArrTime'], axis = 1)))


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [0]:
print(classification_report(test_labels, prediction_lr))
filename = path + '/drive/My Drive/ML Project/classifier_model_lr_ru.sav'
pickle.dump(logreg, open(filename, 'wb'))

              precision    recall  f1-score   support

         0.0       0.94      0.93      0.93    365677
         1.0       0.74      0.78      0.76     97182

    accuracy                           0.90    462859
   macro avg       0.84      0.85      0.85    462859
weighted avg       0.90      0.90      0.90    462859

